Author: Hiranya Jayakody. April 2020.

Code developed for Smart Robotics Viticulture Group, UNSW, Sydney.

Neural Network based on Matterport implementation of Mask-RCNN at https://github.com/matterport/Mask_RCNN

### PART 1: Install Mask-RCNN repo from Matterport

In [0]:
!git clone https://github.com/matterport/Mask_RCNN.git
!pip install -r 'Mask_RCNN/requirements.txt'
!cd Mask_RCNN ; python setup.py install

In [0]:
!pip show mask-rcnn

In [0]:
!pip install tensorflow==1.5.1

In [0]:
!pip install keras==2.1.5

In [0]:
!pip install opencv-python==3.4.9.31

In [0]:
#mount your google drive if necessary
from google.colab import drive
drive.mount('/content/drive')

### PART 2: Set-up Mask-RCNN for inference

In [0]:
# restart runtime if 'No module named 'mrcnn'' error occurs

import os
import cv2
import glob
import sys
import json
import datetime
import numpy as np
import skimage.draw
import imutils
import imgaug
import statistics as st
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

from mrcnn.config import Config
from mrcnn import visualize
from mrcnn import model as modellib, utils
from matplotlib import pyplot as plt

In [0]:
CWD = 'drive/My Drive/Colab Notebooks/'
STOMATA_WEIGHTS_PATH = os.path.join(CWD,'2020_mask_rcnn_stomata_51.h5') 
WEIGHT_FILE_NAME = 'stomata'
CLASS_NAME = 'stomata'
DATASET_DIR = os.path.join(CWD,'images/')
INPUT_IMG_DIR = os.path.join(DATASET_DIR,'test/') 
OUTPUT_IMG_DIR = os.path.join(DATASET_DIR,'results/')

In [0]:
#create config for inference
class InferenceConfig(Config):
    # Set batch size to 1 since we'll be running inference on
    # one image at a time. Batch size = GPU_COUNT * IMAGES_PER_GPU
    NAME = CLASS_NAME #provide a suitable name
    NUM_CLASSES = 1+1 #background+number of classes
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1
    RPN_ANCHOR_SCALES = (12,24,48,96,192) #anchor box scales for the application
    DETECTION_MIN_CONFIDENCE = 0.6 #set min confidence threshold
    DETECTION_MAX_INSTANCES = 500
    POST_NMS_ROIS_INFERENCE = 10000
    IMAGE_MAX_DIM = 1024
    IMAGE_MIN_DIM = 800
    MEAN_PIXEL = np.array([133.774, 133.774, 133.774]) #DEFAULT VALUES FOR STOMATA MODEL. Change as necessary. #matterport takes the input as RGB

In [0]:
#create inference object
inference_config = InferenceConfig()
inference_config.display()

# Load the model in inference mode
model = modellib.MaskRCNN(mode="inference", config=inference_config, model_dir=CWD)
model_path = STOMATA_WEIGHTS_PATH #os.path.join(CWD,'mask_rcnn_stomata.h5')

print("Loading weights from ", model_path)
model.load_weights(model_path, by_name=True)

### PART 3: Apply model to identify stomata

3.1 Function Definitions

In [0]:
# Statistical filter

def stomata_filter(r_pd_):
    #stomata filer: This code filters out stomata like shapes which are of wrong size, using confidence measures.
    high_scores = r_pd_["scores"][r_pd_["scores"]>0.90]
            
    if len(high_scores) > 0:
        percentile_thres = np.min(high_scores)
    else:
        percentile_thres = np.percentile(r_pd_["scores"], 95) #st.median(r_pd_["scores"])
    
    high_conf_areas = r_pd_["areas"][r_pd_["scores"]>=percentile_thres] #conf_threshold
    high_conf_scores = r_pd_["scores"][r_pd_["scores"]>=percentile_thres]
    
    high_conf_avg_area = st.mean(high_conf_areas)
    
    above_avg = high_conf_areas[high_conf_areas>=high_conf_avg_area]
    below_avg = high_conf_areas[high_conf_areas<high_conf_avg_area]
    
    above_avg_conf = high_conf_scores[high_conf_areas>=high_conf_avg_area]
    below_avg_conf = high_conf_scores[high_conf_areas<high_conf_avg_area]
    
    #based on data length
    if len(above_avg) >= len(below_avg):
        optimal_area = np.percentile(above_avg, 50) #(st.mean(above_avg)+np.max(above_avg))/2.0 #can we use percentie 
        st_size = 'LARGE'
    else:
        #smaller elements may not be stomata, so check for their overall confidence with respect to the confidence of larger areas
        if np.mean(above_avg_conf) >= np.mean(below_avg_conf) and len(above_avg) > 1:
            optimal_area = np.percentile(above_avg, 50)
            st_size = 'LARGE'
        elif len(above_avg) <=1 and np.max(above_avg_conf) > 0.985:
            optimal_area = np.percentile(above_avg, 50)
            st_size = 'LARGE'
        else:
            optimal_area = np.percentile(below_avg, 75)
            st_size = 'SMALL'
       
    if st_size == 'LARGE':
        indices_ = r_pd_["scores"][np.logical_and(r_pd_["areas"]> (optimal_area*0.55),r_pd_["areas"]<1.5*optimal_area )].index.values.astype(int)
    else:
        indices_ = r_pd_["scores"][np.logical_and(r_pd_["areas"]> (optimal_area*0.65),r_pd_["areas"]<1.5*optimal_area )].index.values.astype(int)

    return indices_
    



In [0]:
# Other supporting functions

def get_filename(string_):
    #get image filename    
    start = string_.find('test/')+5
    end = string_.find('.jpg',start)
    filename = string_[start:end]
    return filename

def hisEqulColor(img):
    #contrast limited histogram equalisation
    ycrcb=cv2.cvtColor(img,cv2.COLOR_BGR2YCR_CB)
    channels=cv2.split(ycrcb)
    clahe = cv2.createCLAHE(clipLimit=1.0, tileGridSize=(25,25))
    channels[0] = clahe.apply(channels[0])
    cv2.merge(channels,ycrcb)
    cv2.cvtColor(ycrcb,cv2.COLOR_YCR_CB2BGR,img)
    return img

def sharpenColor(img):
    #sharpen image
    kernel_sharpening = np.array([[-1,-1,-1], 
                              [-1, 9,-1],
                              [-1,-1,-1]])
    img = cv2.filter2D(img, -1, kernel_sharpening)
    return img




### PART 3A: Test on a single image ###

Please Refer to Part 3B and remove the loop for the folder.

### PART 3B: Test on an image folder

In [0]:
DATA_PATH = os.path.join(INPUT_IMG_DIR,'*jpg')
files = glob.glob(DATA_PATH)

stomata_data = pd.DataFrame(columns=['filename','num_stomata','scores','areas'])

print(cv2. __version__)

In [0]:
counter = 0
for img in files:
    filename = get_filename(img)
    print(filename)
    image = cv2.imread(img)
    image = imutils.resize(image,width=1024)
    image_original = image
    image = cv2.cvtColor(image,cv2.COLOR_BGR2RGB) #opnecv uses BGR convention
    
    
    #call histogram equalize function (optional)
    image = hisEqulColor(image)
    #image = sharpenColor(image)
    
    #convert to grayscale and save as a three channel jpeg then read it back and convert
    image_gray = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
    cv2.imwrite('current_image.jpg', image_gray)
    
    image_new = cv2.imread('current_image.jpg')
    image = cv2.cvtColor(image_new,cv2.COLOR_BGR2RGB) #opnecv uses BGR convention
    image = imutils.resize(image,width=1024)
    
    #run the image through the model
    print("making predictions with Mask R-CNN...")
    r = model.detect([image], verbose=1)[0]
    
    #create dataframe for ease of use
    r_pd = pd.DataFrame(columns=['class_id','scores', 'areas'])
        
    #create array to store areas
    num_stomata = len(r["scores"])
  
    r_pd["class_ids"] = r["class_ids"]
    r_pd["scores"] = r["scores"]
               
    #retrieve area values from X,Y coordinates
    for i in range(0, len(r_pd["scores"])): 
        (startY,startX,endY,endX) = r["rois"][i]
        r_pd["areas"][i] = abs(startY-endY)*abs(startX-endX)
        
        
    #see how many stomata are on the image
    #1. If there are more than 2 stomata, do the following.
    #2. get the median score for confidence
    #3. get the average area for median and above
    #4. reject areas 90% or less than the average median area
    
    if num_stomata == 0:
        print("no stomata detected")
        stomata_data.loc[counter] = [filename,num_stomata,0]
        counter +=1
        cv2.imwrite(OUTPUT_IMG_DIR+filename+'_000'+'.jpg', image)
        continue
    
    if num_stomata >= 2:
        
        indices = stomata_filter(r_pd)
        #indices = r_pd["scores"][:].index.values.astype(int) #this ignores the statistical filter

    else: 
        indices = [0]
        
    print(indices)
            
    # loop over of the detected object's bounding boxes and masks
    areas = []
    for i in range(0, len(indices)):
        classID = r["class_ids"][indices[i]]
        mask = r["masks"][:, :, indices[i]]
        color = [0,0,0]
        areas.append(np.sum(mask == True))

        #uncomment to visualize the pixel-wise mask of the object
        #image = visualize.apply_mask(image, mask, color, alpha=0.5)
        
        #visualize contours
        mask[mask ==True] = 1
        mask[mask == False] = 0
        mask = mask.astype(np.uint8)
        mask,contours, hierarchy = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
        cv2.drawContours(image_original,contours, 0, [0,255,0], 4)
    
    image = cv2.cvtColor(image,cv2.COLOR_RGB2BGR)

    for i in range(0,len(indices)):
        (startY,startX,endY,endX) = r["rois"][indices[i]]
        classID = r["class_ids"][indices[i]]
        label = classID
        score = r["scores"][indices[i]]
        color = [255,0,0]

        #uncomment to draw bounding box around stomata
        #cv2.rectangle(image_original,(startX,startY),(endX,endY),color,2)

        #uncomment to print confidence value
        #text = "{}: {:.3f}".format(label,score)
        #y = startY - 10 if startY - 10 > 10 else startY + 10
        #cv2.putText(image_original,text,(startX,y),cv2.FONT_HERSHEY_SIMPLEX,0.8,color,2)

    id_str= str(len(indices))
    stomata_data.loc[counter] = [filename,len(indices),r["scores"][indices],areas]
    counter +=1
    cv2.imwrite(OUTPUT_IMG_DIR+filename+'_'+id_str.zfill(3)+'.jpg', image_original)
    

stomata_data.to_csv(OUTPUT_IMG_DIR+'results.csv',encoding='utf-8', index=False)
    
    